In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

In [3]:
import numpy as np
import os
import time

### Read the data

In [4]:
text_path = "/content/drive/MyDrive/Colab Notebooks/NLP/2/10-Text Generator(Attar's Poem)/Sample/naserkhosro.txt"
text = open(text_path, 'rb').read().decode(encoding='utf-8')

In [5]:
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 633429 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

ای قبهٔ گردندهٔ بی روزن خضرا
با قامت فرتوتی و با قوت برنا
فرزند توایم ای فلک، ای مادر بدمهر
ای مادر ما چونکه همی کین کشی از ما؟
فرزند تو این تیره تن خامش خاکی است
پاکیزه خرد نیست نه این جوهر گویا
تن خانهٔ این گوهر والای شریف است
تو مادر این خانهٔ این


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

50 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [10]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '(' :   3,
  ')' :   4,
  '.' :   5,
  ':' :   6,
  '«' :   7,
  '»' :   8,
  '،' :   9,
  '؛' :  10,
  '؟' :  11,
  'ء' :  12,
  'آ' :  13,
  'ؤ' :  14,
  'ئ' :  15,
  'ا' :  16,
  'ب' :  17,
  'ة' :  18,
  'ت' :  19,
  ...
}


In [11]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:30]), text_as_int[:30]))

'ای قبهٔ گردندهٔ بی روزن خضرا\nب' ---- characters mapped to int ---- > [16 49  1 37 17 41 43  1 48 26 24 40 24 41 43  1 17 49  1 26 42 27 40  1
 23 31 26 16  0 17]


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [12]:
# The maximum length sentence we want for a single input in characters
seq_length = 100

In [13]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [17]:
for i in char_dataset.take(10):
    print(i.numpy(), idx2char[i.numpy()])

16 ا
49 ی
1  
37 ق
17 ب
41 ه
43 ٔ
1  
48 گ
26 ر


### Sliding Window for Sequence Generation

Instead of using simple batching, we use the **sliding window** method to generate overlapping sequences of length `seq_length + 1`.  
This ensures that every character in the text contributes to multiple training samples, preserving the continuity of the text.  
Compared to fixed-size batching, this approach creates **richer and more context-aware training data**, helping the model better capture character-level dependencies in the text.


In [21]:
# Sliding window: ساخت پنجره‌های طول ثابت با گام 1
sequences = char_dataset.window(size=seq_length + 1, shift=1, drop_remainder=True)

In [22]:
# هر پنجره را به یک تانسور تبدیل می‌کنیم تا بتوان پردازشش کرد
sequences = sequences.flat_map(lambda window: window.batch(seq_length + 1))

For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [24]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [25]:
# بررسی ۳ نمونه اولیه
for input_example, target_example in dataset.take(3):
    print('Input: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target:', repr(''.join(idx2char[target_example.numpy()])))
    print('-' * 30)

Input:  'ای قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر '
Target: 'ی قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر م'
------------------------------
Input:  'ی قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر م'
Target: ' قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر ما'
------------------------------
Input:  ' قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر ما'
Target: 'قبهٔ گردندهٔ بی روزن خضرا\nبا قامت فرتوتی و با قوت برنا\nفرزند توایم ای فلک، ای مادر بدمهر\nای مادر ما '
------------------------------
